In [ ]:
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# функция для раздулки
def get_razd(sx_razd,v_):
    ch_razd = {}
    #     если это деловая часть
    for ind in sx_razd.index:

        if sx_razd.at[ind,'Из какой части пром']==0:
            v___ = v_ * sx_razd.at[ind,'Процент разделки']
    #         df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':fff.at[ind,'Часть'],'Объем':v___}))
            ch_razd[sx_razd.at[ind,'Часть']] =v___


    for ind in sx_razd.index:
    #     для пром
        if sx_razd.at[ind,'Из какой части пром']!=0:
    #         print(fff.at[ind,'Из какой части пром'])
    #         Процент вет брака
            pr_vetbr = sx_razd[(sx_razd['Из какой части пром']==sx_razd.at[ind,'Из какой части пром'])&(sx_razd['Часть']==sx_razd.at[ind,'Часть'])]['Процент разделки'].values
            v_pochasti = ch_razd[sx_razd.at[ind,'Из какой части пром']]
            it_prom = pr_vetbr * v_pochasti
            ch_razd[sx_razd.at[ind,'Часть']] = it_prom
            ch_razd[sx_razd.at[ind,'Из какой части пром']] = ch_razd[sx_razd.at[ind,'Из какой части пром']] - ch_razd[sx_razd.at[ind,'Часть']]
    
#     Выбираем минимальное из деловых частей (там где больше одного элемента)
    for ind in sx_razd.index:
        ch_razd[sx_razd.at[ind,'Часть']] = min(ch_razd[sx_razd.at[ind,'Часть']])
    
    for key___ in ch_razd.keys():
        ch_razd[key___] = ch_razd[key___].sum()
    return ch_razd


# делит по 100 гр
def get_po_sto_gr(inp):
    inp = inp.set_index('Дата забоя')
    df = pd.DataFrame(inp.index)
    df = df.set_index('Дата забоя')
    for ind, col in enumerate(inp.columns):
#         print(col.split('-')[0]=='2,500')
        try:
            if col.split('-')[0]=='2,500':
                df['2,500-3,500'] = inp['2,500-3,500']
#                 print(col)
            
            if ind ==0 or ind%2==0:
                cols = inp.iloc[:,[ind,ind+1]].columns
                df[cols[0].split('-')[0]+'-'+cols[1].split('-')[1]] = inp.iloc[:,[ind,ind+1]].sum(axis = 1)

            
        except IndexError:
            pass
    return df.reset_index()

# логика формирования итогового фрейма

def get_cons_arr(df_gr,dict_,chema_r,dict_sx):

    df_cons = pd.DataFrame()
    for date in df_gr['Дата забоя'].unique():

        df = df_gr[df_gr['Дата забоя']==date]

        for grad in dict_.keys():


            v_razd = df[grad].values * dict_[grad]
            v_cb = df[grad].values * (1- dict_[grad])

    #         Часть с разделкой
            if v_razd>0:

    #             print(v_razd)

    #             Иду по пропорции разделки
                for el_sx in dict_sx.keys():

    #                 Объем по схеме
                    v_ = v_razd * dict_sx[el_sx]
    #                 print(str(v_razd)+' '+ str(v_))
                    sx_razd = chema_r[chema_r['Номер'] == el_sx]
                    it_chema = get_razd(sx_razd,v_)
                    for key_ in it_chema.keys():
                        df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':key_,'Объем':it_chema[key_]}))
    #                     print(key_)
    #                     break
    #                 break



    #                 print('общий объем'+ str(v_razd) + ' Объем на конкретную схему эту '+str(v_))



    #         Часть ЦБ
            if v_cb>0:

                df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':'ЦБ','Объем':v_cb}))
    return df_cons


# разбивает часть по процентц в аргументе
def calc_division_ch(cons2,from_name,to_name,percent):
    for_ch_okor = cons2.loc[cons2['Наименование части']==from_name]
    ost_okor = cons2.loc[cons2['Наименование части']==from_name]

    for col in for_ch_okor.columns:
        try:
            for_ch_okor.loc[:,col] = for_ch_okor.loc[:,col]*percent
        except TypeError:
            pass

    for col in ost_okor.columns:
        if ost_okor[col].dtype=='float':
            cons2.loc[cons2['Наименование части']==from_name,col] = ost_okor[col]-for_ch_okor[col]

    for_ch_okor['Наименование части'] = to_name

    cons2 = cons2.append(for_ch_okor).reset_index(drop=True)
    return cons2
        
    
# формирование чахохбили
# 

# суммирует значения с фильтром по числовому значению
def total_sum(name_col,value_,df):
    sum_ = 0
    for str_ in df[df[name_col]==value_].sum():
        if isinstance(str_, np.float64):
            sum_ = sum_+str_

    return sum_


# Формирование чахохбили из объема

"""Параметры : cons2:обрабобатываемый df
               from_name: часть откуда бурем 
               to_name: часть куда транспортируем объем
               percent: процент по анатомии
               tagret_val: целевой объем to_name"""

def calc_dop_chakh(cons2,from_name,to_name,percent,tagret_val):
    
    target_calc = tagret_val*percent
    
    for_ch_okor = cons2.loc[cons2['Наименование части']==from_name]
    ost_okor = cons2.loc[cons2['Наименование части']==from_name]
    
    tot_sum = total_sum('Наименование части',from_name,cons2)
    if target_calc >= tot_sum:
        print('Не возможно сделать данный объем '+to_name+' из '+from_name+', уменьшите его')
        return 0
    for col in for_ch_okor.columns:
        if for_ch_okor[col].dtype=='float':
            for_ch_okor.loc[cons2['Наименование части']==from_name,col] = for_ch_okor.loc[cons2['Наименование части']==from_name,col]/tot_sum*target_calc


    for col in ost_okor.columns:
            if ost_okor[col].dtype=='float':
                cons2.loc[cons2['Наименование части']==from_name,col] = ost_okor[col]-for_ch_okor[col]

    for_ch_okor['Наименование части'] = 'Сырье для чахохбили'
    
    cons2 = cons2.append(for_ch_okor).reset_index(drop=True)
    
    return cons2

In [ ]:
directory = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\analyze_zab\\unput\\'
directory_out = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\analyze_zab\\output\\'

year = 2021

# какие месяца по дням а какие по месяцам
params = {'days':[8],'month':[9,10,11,12]}

# Сколько с градации брать на разделку


dict_ ={'0,500-0,600':0, 
        '0,600-0,700':0, 
        '0,700-0,800':0, 
        '0,800-0,900':0,
        '0,900-1,000':0.9, 
        '1,000-1,100':0.9, 
        '1,100-1,200':0.9, 
        '1,200-1,300':0.9,
        '1,300-1,400':0.6, 
        '1,400-1,500':0.9, 
        '1,500-1,600':0.9, 
        '1,600-1,700':0,
        '1,700-1,800':0, 
        '1,800-1,900':0, 
        '1,900-2,000':0, 
        '2,000-2,100':0,
        '2,100-2,200':0, 
        '2,200-2,300':0, 
        '2,300-2,400':0, 
        '2,400-2,500':0,
        '2,500-3,500':0}

cols_sub = ['Дата забоя','Печень, тн','Сердца, тн','Желудки, тн','Шеи, тн','Жир','Головы, тн','Лапы, тн','Утиль, тн']



# пропорция по схемам разделки
dict_sx = {1:0.2,2:0.3,3:0.3,4:0.1,5:0.1}

# Сколько сделать чахохбили
target_chakhoh = 6

In [ ]:
ukpf =  pd.read_excel(directory+'УКПФ_заб.xlsx',sheet_name='Забой')
ukpf_sub = ukpf[cols_sub].copy()
ukpf = get_po_sto_gr(ukpf)
chema_r = pd.read_excel(directory+'Схемы разделки УКПФ.xlsx',sheet_name='Лист1')

In [ ]:
# Часть по дням
df_for_days = ukpf[([True  if x.month == params['days'][0] else False for x in ukpf['Дата забоя']])&(ukpf['Дата забоя'].dt.year==year)]
df_for_days_gr = df_for_days.groupby('Дата забоя').sum().reset_index()
# для суб
df_for_days_sub = ukpf_sub[([True  if x.month == params['days'][0] else False for x in ukpf_sub['Дата забоя']])&(ukpf_sub['Дата забоя'].dt.year==year)]
df_for_days_gr_sub = df_for_days_sub.groupby('Дата забоя').sum().reset_index()

In [ ]:
df_for_days = get_cons_arr(df_for_days_gr,dict_,chema_r,dict_sx)

In [ ]:
# Часть по месяцам
df_for_month = ukpf[ukpf['Дата забоя'].dt.month.isin(params['month'])&(ukpf['Дата забоя'].dt.year==year)]
df_for_month['Дата забоя'] = [datetime.datetime(x.year,x.month,1) for x in df_for_month['Дата забоя']]
df_for_month_gr = df_for_month.groupby('Дата забоя').sum().reset_index()

# для суб

df_for_month_sub = ukpf_sub[ukpf_sub['Дата забоя'].dt.month.isin(params['month'])&(ukpf_sub['Дата забоя'].dt.year==year)]
df_for_month_sub['Дата забоя'] = [datetime.datetime(x.year,x.month,1) for x in df_for_month_sub['Дата забоя']]
df_for_month_gr_sub = df_for_month_sub.groupby('Дата забоя').sum().reset_index()

In [ ]:
sub_stack_month = df_for_month_gr_sub.set_index('Дата забоя').stack().reset_index()
sub_stack_days = df_for_days_gr_sub.set_index('Дата забоя').stack().reset_index()
cons_sub = sub_stack_month.append(sub_stack_days)
cons_sub.columns = ['Дата забоя','Наименование части','Объем']

In [ ]:
df_for_month = get_cons_arr(df_for_month_gr,dict_,chema_r,dict_sx)
cons1 = df_for_days.append(df_for_month)
cons1 = cons_sub.append(cons1)
cons1.loc[cons1['Наименование части']!='ЦБ','Градация'] = ''
cons2 = pd.pivot_table(cons1, values = 'Объем',index =['Градация','Наименование части'],columns = 'Дата забоя',aggfunc=np.sum ).reset_index().fillna(0)

In [ ]:
# cons2 = calc_dop_chakh(cons2,'Окорочек','Сырье для чахохбили',0.7)
# cons2 = calc_dop_chakh(cons2,'Грудка','Сырье для чахохбили',0.3)
cons2 = calc_dop_chakh(cons2,'Окорочек','Сырье для чахохбили',0.7,target_chakhoh)
cons2 = calc_dop_chakh(cons2,'Грудка','Сырье для чахохбили',0.3,target_chakhoh)
cons2 = cons2.groupby(['Градация','Наименование части']).sum().reset_index()

In [ ]:
cons2.sort_values(['Наименование части','Градация']).to_excel(directory_out+' testUKPF.xlsx')

In [ ]:
gr = pd.concat([cons2['Наименование части'],cons2[[x for x in cons2.columns if cons2[x].dtype=='float']].sum(axis = 1)],axis = 1)
gr.columns = ['Наименование части','Объем']
gr = gr.groupby('Наименование части').sum().reset_index()
gr.sort_values(['Наименование части']).to_excel(directory_out+' testUKPFgr.xlsx')

In [ ]:
cons2[cons2['Наименование части']=='Окорочек']

In [ ]:
for col in cons2.columns:
    if cons2[col].dtype=='float':
        print(cons2.loc[cons2['Наименование части']=='Окорочек',col])

In [ ]:
cons2[cons2['Наименование части']=='Окорочек']

In [ ]:
# суммирует значения с фильтром по числовому значению
def total_sum(name_col,value_,df):
    sum_ = 0
    for str_ in df[df[name_col]==value_].sum():
        if isinstance(str_, np.float64):
            sum_ = sum_+str_

    return sum_

In [ ]:
total_sum('Наименование части','Окорочек',cons2)

In [ ]:
ch_target = 5

In [ ]:
okor_target = ch_target*0.7

In [ ]:
okor_target

In [ ]:
# формирование чахохбили
# 
"""Параметры : cons2:обрабобатываемый df
               from_name: часть откуда бурем 
               to_name: часть куда транспортируем объем
               percent: процент по анатомии
               tagret_val: целевой объем to_name"""

def calc_dop_chakh(cons2,from_name,to_name,percent,tagret_val):
    
    target_calc = tagret_val*percent
    
    for_ch_okor = cons2.loc[cons2['Наименование части']==from_name]
    ost_okor = cons2.loc[cons2['Наименование части']==from_name]
    
    tot_sum = total_sum('Наименование части',from_name,cons2)
    if target_calc >= tot_sum:
        print('Не возможно сделать данный объем '+to_name+' из '+from_name+', уменьшите его')
        return 0
    for col in for_ch_okor.columns:
        if for_ch_okor[col].dtype=='float':
            for_ch_okor.loc[cons2['Наименование части']==from_name,col] = for_ch_okor.loc[cons2['Наименование части']==from_name,col]/tot_sum*target_calc


    for col in ost_okor.columns:
            if ost_okor[col].dtype=='float':
                cons2.loc[cons2['Наименование части']==from_name,col] = ost_okor[col]-for_ch_okor[col]

    for_ch_okor['Наименование части'] = 'Сырье для чахохбили'
    
    cons2 = cons2.append(for_ch_okor).reset_index(drop=True)
    
    return cons2

In [ ]:
calc_dop_chakh(cons2,'Окорочек','Сырье для чахохбили',0.7,5)

In [ ]:
for_ch_okor

In [ ]:
lst

In [ ]:
cons2.loc[cons2['Наименование части']=='Окорочек',col]

In [ ]:
# формирование чахохбили
def calc_dop_chakh(cons2,from_name,to_name,percent):
    for_ch_okor = cons2.loc[cons2['Наименование части']==from_name]
    ost_okor = cons2.loc[cons2['Наименование части']==from_name]

    for col in for_ch_okor.columns:
        try:
            for_ch_okor.loc[:,col] = for_ch_okor.loc[:,col]*percent
        except TypeError:
            pass

    for col in ost_okor.columns:
        if ost_okor[col].dtype=='float':
            cons2.loc[cons2['Наименование части']==from_name,col] = ost_okor[col]-for_ch_okor[col]

    for_ch_okor['Наименование части'] = to_name

    cons2 = cons2.append(for_ch_okor).reset_index(drop=True)
    return cons2